# Data Pre Processing

This is a notebook for intaking a raw data file, preprocessing it, and exporting it in a format in which it can easily be labeled.

In [63]:
import pandas as pd

In [64]:
df = pd.read_csv("data/directoryident.tsv", sep="\t")
df.head()

,dn,lastname,givenname,middle,dob,address
0,"uid=madanafl,ou=People,dc=umich,dc=edu",Madanay,Farrah,L,1990091200Z,NaN
1,"uid=jocbran,ou=People,dc=umich,dc=edu",Branham,Jocelyn,NaN,20050616000000Z,{addr1=345A Infantry Ter}:{city=San Francisco}...
2,"uid=stoicabr,ou=People,dc=umich,dc=edu",Stoica,Brooke,NaN,20031007000000Z,{addr1=9051 Salem Farms Drive}:{city=South Lyo...
3,"uid=heavens,ou=People,dc=umich,dc=edu",Smith,Heaven,NaN,19880104000000Z,{addr1=6655 Jackson Rd Unit 191}:{city=Ann Arb...
4,"uid=funnim,ou=People,dc=umich,dc=edu",Funni,Matthew,NaN,1989121900Z,{addr1=2095 pauls way}:{city=Commerce twp}:{st...


## Preprocessing

### Cleanup

In [65]:
# Add unique ID column
# Uses the row number of original csv file as unique ID
df['Unique ID'] = df.index + 1

In [66]:
# Turn all NaN values into empty strings
df['address'] = df['address'].fillna(" ")
df['middle'] = df['middle'].fillna(" ")

In [67]:
df.head()

,dn,lastname,givenname,middle,dob,address,Unique ID
0,"uid=madanafl,ou=People,dc=umich,dc=edu",Madanay,Farrah,L,1990091200Z,,1
1,"uid=jocbran,ou=People,dc=umich,dc=edu",Branham,Jocelyn,,20050616000000Z,{addr1=345A Infantry Ter}:{city=San Francisco}...,2
2,"uid=stoicabr,ou=People,dc=umich,dc=edu",Stoica,Brooke,,20031007000000Z,{addr1=9051 Salem Farms Drive}:{city=South Lyo...,3
3,"uid=heavens,ou=People,dc=umich,dc=edu",Smith,Heaven,,19880104000000Z,{addr1=6655 Jackson Rd Unit 191}:{city=Ann Arb...,4
4,"uid=funnim,ou=People,dc=umich,dc=edu",Funni,Matthew,,1989121900Z,{addr1=2095 pauls way}:{city=Commerce twp}:{st...,5


In [68]:
# Convert all columns to string
df = df.astype(str)

In [69]:
# take sample of 5000
df = df.sample(n=500)

In [70]:
# Function to parse a singular dn field
# Returns the uid in the dn
def parse_dn(dn):
    dn = dn.split(',')
    dn = [x.split('=') for x in dn]
    dn = {x[0].strip(): x[1].strip() for x in dn}
    return dn["uid"]

In [71]:
# Function to concatenate every value in the address 
# We want to concatenate all values inside curly braces
def concat_address(address):
    if address != ' ':
        address = address.split('}:{')
        try: 
            address = [x.split('=')[1] for x in address]
        except:
            return "DELETE"
        address = ' '.join(address)
    return address

In [72]:
# Use vectorization to set new values for dn and address columns
df['dn'] = df['dn'].apply(parse_dn)
df['address'] = df['address'].apply(concat_address)
df['dob'] = df['dob'].apply(lambda x : x[:8])

In [73]:
if "DELETE" in df['address'].values:
    df = df[df['address'] != "DELETE"]

df.head()

,dn,lastname,givenname,middle,dob,address,Unique ID
405977,rdrummon,Drummond,Roger,Eugene,19691201,935 Oakes St East Palo Alto CA 94303-2555 UNIT...,405978
395083,shining,Donnell,Kira,Ann,19830301,624 Florence St Daly City CA 94014-2820 UNITED...,395084
149615,siriyar,Yarlagadda,Siri,,20010206,16073 Johnson Creek Dr Northville MI 48168-800...,149616
262381,michaehe,Heinrich,Michael,Timothykarl,19940817,2267 Medford Rd. Ann Arbor MI 48104 UNITED STA...,262382
70744,raynao,Olver,Rayna,Hope,20040803,17660 Loveland St Livonia MI 48152-3241 UNITED...,70745


In [74]:
# Change column names
df.columns = ['ID', 'Last Name', 'First Name', 'Middle Name', 'Date of Birth', 'Address', 'Unique ID']
# Reorder columns
df = df[['Unique ID', 'ID', 'First Name', 'Middle Name', 'Last Name', 'Date of Birth', 'Address']]

In [75]:
df.head()

,Unique ID,ID,First Name,Middle Name,Last Name,Date of Birth,Address
405977,405978,rdrummon,Roger,Eugene,Drummond,19691201,935 Oakes St East Palo Alto CA 94303-2555 UNIT...
395083,395084,shining,Kira,Ann,Donnell,19830301,624 Florence St Daly City CA 94014-2820 UNITED...
149615,149616,siriyar,Siri,,Yarlagadda,20010206,16073 Johnson Creek Dr Northville MI 48168-800...
262381,262382,michaehe,Michael,Timothykarl,Heinrich,19940817,2267 Medford Rd. Ann Arbor MI 48104 UNITED STA...
70744,70745,raynao,Rayna,Hope,Olver,20040803,17660 Loveland St Livonia MI 48152-3241 UNITED...


### Similarity Measures

In [76]:
# Sample 200 records from df into new df
df_sample = df.sample(100)

In [77]:
df_sample.head()

,Unique ID,ID,First Name,Middle Name,Last Name,Date of Birth,Address
265875,265876,mmchonic,Milan,M,Chonich,19440701,5783 Glasgow Dr Troy MI 48085-3139 UNITED STAT...
136564,136565,ykauff,Yannick,,Kauffmann,19990209,J.F. Debeckerlaan 93 Brussels 1200 BELGIUM BEL}
57236,57237,yesha,Yesha,Vijaykumar,Bhavsar,20030902,L-303 Shukan Residency Near Vandematram Arcade...
37529,37530,vshishka,Vasiliy,,Shishkarev,19921031,1520 Bonnie Place Mount Vernon WA 98274 UNITED...
276416,276417,dgildin,Daniel,,Gildin,19911208,Rua Desembargador do Vale 64 apt 102 Sao Paulo...


In [78]:
# Remove sampled records from df
df = df.drop(df_sample.index)

With a sample df, we can now build a similarity measure df by comparing each record in the sample df with the other records.

In [79]:
import Levenshtein as lev

In [80]:
"""
Returns normalized levenshtein distance between two strings
"""
def levenshtein_distance(
        s1: str,
        s2: str
) -> int:
    distance = lev.distance(s1, s2)
    try:
        return 1 - distance / float(max(len(s1), len(s2)))
    except ZeroDivisionError:
        return 0

In [81]:
"""
Returns Levenshtein distance between each field of two rows
Assumes following format for rows:
    row = pd.Series([First Name, Middle Name, Last Name, DOB, Address, ID])
"""
def row_similarity(
        row_1: pd.Series, 
        df_2: pd.DataFrame
    ) -> pd.DataFrame:

    # Compute similarity measures for each column using Levenshtein distance
    first_name_similarity = df_2['First Name'].apply(lambda x : levenshtein_distance(row_1['First Name'], x))
    middle_name_similarity = df_2['Middle Name'].apply(lambda x : levenshtein_distance(row_1['Middle Name'], x))
    last_name_similarity = df_2['Last Name'].apply(lambda x : levenshtein_distance(row_1['Last Name'], x))
    dob_similarity = df_2['Date of Birth'].apply(lambda x : levenshtein_distance(row_1['Date of Birth'], x))
    address_similarity = df_2['Address'].apply(lambda x : levenshtein_distance(row_1['Address'], x))
    id_similarity = df_2['ID'].apply(lambda x : levenshtein_distance(row_1['ID'], x))

    # Return DataFrame with similarity measures
    return pd.DataFrame({
        'UNIQ_ID1': row_1['Unique ID'],
        'UNIQ_ID2': df_2['Unique ID'],
        'ID1': row_1['ID'],
        'ID2': df_2['ID'],
        'ID Similarity': id_similarity,
        'First Name 1' : row_1['First Name'],
        'First Name 2' : df_2['First Name'],
        'First Name Similarity': first_name_similarity,
        'Middle Name 1' : row_1['Middle Name'],
        'Middle Name 2' : df_2['Middle Name'],
        'Middle Name Similarity': middle_name_similarity,
        'Last Name 1' : row_1['Last Name'],
        'Last Name 2' : df_2['Last Name'],
        'Last Name Similarity': last_name_similarity,
        'Date of Birth 1' : row_1['Date of Birth'],
        'Date of Birth 2' : df_2['Date of Birth'],
        'Date of Birth Similarity': dob_similarity,
        'Address 1' : row_1['Address'],
        'Address 2' : df_2['Address'],
        'Address Similarity': address_similarity
    })


In [82]:
"""
Builds similarity measure between records in two df
Creates a new df from the two df with the following columns:
    - ID1: ID of record in df1
    - ID2: ID of record in df2
    - First Name Similarity: Normalized levenshtein distance between first names
    - Middle Name Similarity: Normalized levenshtein distance between middle names
    - Last Name Similarity: Normalized levenshtein distance between last names
    - Date of Birth Similarity: Normalized levenshtein distance between dates of birth
    - Address Similarity: Normalized levenshtein distance between addresses
    - ID Similarity: Normalized levenshtein distance between IDs
"""
def build_similarity_df (
        df_1: pd.DataFrame,
        df_2: pd.DataFrame
) -> pd.DataFrame:
    new_df = pd.DataFrame(columns=[ 'UNIQ_ID1',
                                    'UNIQ_ID2',
                                    'ID1', 
                                   'ID2', 
                                   'ID Similarity',
                                   'First Name 1',
                                   'First Name 2',
                                   'First Name Similarity', 
                                   'Middle Name 1',
                                    'Middle Name 2',
                                   'Middle Name Similarity',
                                    'Last Name 1',
                                    'Last Name 2', 
                                   'Last Name Similarity', 
                                    'Date of Birth 1',
                                    'Date of Birth 2',
                                   'Date of Birth Similarity', 
                                    'Address 1',
                                    'Address 2',
                                   'Address Similarity'])

    # Convert ID columns to string
    df_1["ID"] = df_1["ID"].astype(str)
    df_2["ID"] = df_2["ID"].astype(str)

    def apply_row_similarity(row, new_df):
        sim = row_similarity(row, df_2)
        new_df = pd.concat([new_df, sim], ignore_index=True)
        return new_df

    # Use vectorization to compute similarity between each row in df_1 and df_2
    new_df = df_1.apply(apply_row_similarity, args=(new_df,), axis=1).reset_index(drop=True)

    # New df is a series of dfs, so we need to concatenate them
    new_df = pd.concat(new_df.to_list(), ignore_index=True)

    return new_df

In [83]:
similarity_df = build_similarity_df(df_sample,  df)
similarity_df.head()

,UNIQ_ID1,UNIQ_ID2,ID1,ID2,ID Similarity,First Name 1,First Name 2,First Name Similarity,Middle Name 1,Middle Name 2,Middle Name Similarity,Last Name 1,Last Name 2,Last Name Similarity,Date of Birth 1,Date of Birth 2,Date of Birth Similarity,Address 1,Address 2,Address Similarity
0,265876,405978,mmchonic,rdrummon,0.000,Milan,Roger,0.000000,M,Eugene,0.0,Chonich,Drummond,0.000000,19440701,19691201,0.500,5783 Glasgow Dr Troy MI 48085-3139 UNITED STAT...,935 Oakes St East Palo Alto CA 94303-2555 UNIT...,0.466667
1,265876,395084,mmchonic,shining,0.250,Milan,Kira,0.400000,M,Ann,0.0,Chonich,Donnell,0.142857,19440701,19830301,0.625,5783 Glasgow Dr Troy MI 48085-3139 UNITED STAT...,624 Florence St Daly City CA 94014-2820 UNITED...,0.482759
2,265876,149616,mmchonic,siriyar,0.000,Milan,Siri,0.200000,M,,0.0,Chonich,Yarlagadda,0.000000,19440701,20010206,0.250,5783 Glasgow Dr Troy MI 48085-3139 UNITED STAT...,16073 Johnson Creek Dr Northville MI 48168-800...,0.515152
3,265876,262382,mmchonic,michaehe,0.375,Milan,Michael,0.428571,M,Timothykarl,0.0,Chonich,Heinrich,0.500000,19440701,19940817,0.500,5783 Glasgow Dr Troy MI 48085-3139 UNITED STAT...,2267 Medford Rd. Ann Arbor MI 48104 UNITED STA...,0.481481
4,265876,70745,mmchonic,raynao,0.125,Milan,Rayna,0.000000,M,Hope,0.0,Chonich,Olver,0.000000,19440701,20040803,0.375,5783 Glasgow Dr Troy MI 48085-3139 UNITED STAT...,17660 Loveland St Livonia MI 48152-3241 UNITED...,0.551724


### Discretization

### WPKID

In [84]:
wpkid_df = similarity_df.copy()

In [85]:
# holds bin ranges for each similarity attribute
bin_ranges = {'First Name Similarity' : [],
        'Middle Name Similarity' : [],
        'Last Name Similarity' : [],
        'Date of Birth Similarity' : [],
        'Address Similarity' : [],
        'ID Similarity' : []}

In [86]:
#for solving systems of equations
import sympy as sp 
import math


t = # of intervals<br>
s = interval size<br>
n = # of instances in training dataset<br>

s * t = n<br>
s - 30 = t<br>


In [87]:
# WPKID function to solve for s and t given n, returns pair {s, t}
def interval_size_and_num(n) -> tuple:
    s, t = sp.symbols('s t')
    equation1 = sp.Eq(s * t, n)
    equation2 = sp.Eq(s - 30, t)  
    solution = sp.solve((equation1, equation2), (s, t))
    s_val = solution[0][1] * -1
    t_val = solution[1][1]

    s_int = math.floor(s_val)
    t_int = math.floor(t_val)
    return s_int, t_int

In [88]:
# function that takes in a column and discretizes it, saving bin ranges
def discretize_column(column, df):
    # separate zeros from nonzeros
    zeros_df = wpkid_df[wpkid_df[column] == 0].copy()
    nonzeros_df = wpkid_df[wpkid_df[column] != 0].copy()

    #get num_instances and num_intervals for nonzeros
    num_instances = interval_size_and_num(len(nonzeros_df))[0]
    num_intervals = interval_size_and_num(len(nonzeros_df))[1]

    # get bin ranges for nonzeros
    nonzeros_df[column], bin = pd.qcut(nonzeros_df[column], 
                                        q=num_intervals, 
                                        labels=False,
                                        retbins=True,
                                        duplicates='drop') 
    
    #add 1 to every value to switch to index 1
    nonzeros_df[column] = nonzeros_df[column] + 1

    #save bin ranges
    bin_ranges[column].extend(bin.tolist())

    #recombine zero and nonzero dataframes
    df.loc[df[column] == 0, column] = zeros_df[column]
    df.loc[df[column] != 0, column] = nonzeros_df[column] 

    return df

In [89]:
wpkid_df = discretize_column('First Name Similarity', wpkid_df)
wpkid_df = discretize_column('Middle Name Similarity', wpkid_df)
wpkid_df = discretize_column('Last Name Similarity', wpkid_df)
wpkid_df = discretize_column('Date of Birth Similarity', wpkid_df)
wpkid_df = discretize_column('Address Similarity', wpkid_df)
wpkid_df = discretize_column('ID Similarity', wpkid_df)

In [90]:
"""insert 0 at the beginning of each bin range"""
for key in bin_ranges:
    bin_ranges[key].insert(0, 0)

In [91]:
wpkid_df.head()

,UNIQ_ID1,UNIQ_ID2,ID1,ID2,ID Similarity,First Name 1,First Name 2,First Name Similarity,Middle Name 1,Middle Name 2,Middle Name Similarity,Last Name 1,Last Name 2,Last Name Similarity,Date of Birth 1,Date of Birth 2,Date of Birth Similarity,Address 1,Address 2,Address Similarity
0,265876,405978,mmchonic,rdrummon,0.0,Milan,Roger,0.0,M,Eugene,0.0,Chonich,Drummond,0.0,19440701,19691201,3.0,5783 Glasgow Dr Troy MI 48085-3139 UNITED STAT...,935 Oakes St East Palo Alto CA 94303-2555 UNIT...,101.0
1,265876,395084,mmchonic,shining,4.0,Milan,Kira,17.0,M,Ann,0.0,Chonich,Donnell,7.0,19440701,19830301,4.0,5783 Glasgow Dr Troy MI 48085-3139 UNITED STAT...,624 Florence St Daly City CA 94014-2820 UNITED...,113.0
2,265876,149616,mmchonic,siriyar,0.0,Milan,Siri,8.0,M,,0.0,Chonich,Yarlagadda,0.0,19440701,20010206,1.0,5783 Glasgow Dr Troy MI 48085-3139 UNITED STAT...,16073 Johnson Creek Dr Northville MI 48168-800...,131.0
3,265876,262382,mmchonic,michaehe,7.0,Milan,Michael,18.0,M,Timothykarl,0.0,Chonich,Heinrich,21.0,19440701,19940817,3.0,5783 Glasgow Dr Troy MI 48085-3139 UNITED STAT...,2267 Medford Rd. Ann Arbor MI 48104 UNITED STA...,112.0
4,265876,70745,mmchonic,raynao,1.0,Milan,Rayna,0.0,M,Hope,0.0,Chonich,Olver,0.0,19440701,20040803,2.0,5783 Glasgow Dr Troy MI 48085-3139 UNITED STAT...,17660 Loveland St Livonia MI 48152-3241 UNITED...,148.0


In [92]:
for key, value in bin_ranges.items():
    print(key, value)

First Name Similarity [0, 0.07692307692307687, 0.09090909090909094, 0.09999999999999998, 0.11111111111111116, 0.125, 0.1428571428571429, 0.16666666666666663, 0.18181818181818177, 0.19999999999999996, 0.2222222222222222, 0.23076923076923073, 0.25, 0.2727272727272727, 0.2857142857142857, 0.30000000000000004, 0.33333333333333337, 0.375, 0.4, 0.4285714285714286, 0.4444444444444444, 0.5, 0.625, 1.0]
Middle Name Similarity [0, 0.06666666666666665, 0.09090909090909094, 0.09999999999999998, 0.11111111111111116, 0.125, 0.1428571428571429, 0.16666666666666663, 0.19999999999999996, 0.2222222222222222, 0.25, 0.27908825868008214, 0.2857142857142857, 0.33333333333333337, 0.375, 0.4, 0.4285714285714286, 0.5, 1.0]
Last Name Similarity [0, 0.05882352941176472, 0.07692307692307687, 0.08333333333333337, 0.09090909090909094, 0.09999999999999998, 0.11111111111111116, 0.125, 0.1428571428571429, 0.15384615384615385, 0.16666666666666663, 0.18181818181818177, 0.19999999999999996, 0.2222222222222222, 0.23076923

In [93]:
# export wpkid_df to csv 
wpkid_df.to_csv('data/sampleunlabeledident.csv', index=False)